In [1]:
# imports
# create Fully connected NN
# set device
# Hyperparameters
# load data
# initialise network
# loss and regularizer
# train
# check accuracy on train and test

In [6]:
# imports
import torch
import torch.nn as nn # Neural Network modules
import torch.optim as optim # all optimisation algos like SGD, Adam, etc
import torch.nn.functional as F # all functions that dont have parameters liek relu, tanh
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms # this has transformations we can perform on our dataset

In [10]:
# create Fully connected NN

class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN,self).__init__() # super calls initialization method of parent class i.e. nn.Module in this case
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
# check if it works by randomly creating an input
model = NN(784, 10)
x = torch.rand(64, 784)
print(x.shape)
y = model(x)
print(y.shape)
print(y[0]) # we can see since we hadn't called the softmax the sum of 10 values is not equal to 1



torch.Size([64, 784])
torch.Size([64, 10])
tensor([ 0.0976,  0.1003, -0.0958,  0.1367,  0.2317, -0.2072, -0.0990, -0.1688,
         0.0081,  0.1027], grad_fn=<SelectBackward0>)


In [11]:
# set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: ', device)


device:  cpu


In [17]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

In [52]:
# load data
train_dataset = datasets.MNIST(root = 'dataset/', train=True,transform = transforms.ToTensor(), download= True)
train_loader = DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = 'dataset/', train=False,transform = transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size= batch_size, shuffle = True)

In [56]:
# initialise network
model = NN(input_size = input_size, num_classes= num_classes).to(device)
model


NN(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [57]:
# loss and regularizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [58]:
# train
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device = device)
        targets = targets.to(device = device)
        #print(data.shape, targets.shape) # [64,1,28,28]
        
        # Get to correct shape
        data = data.view(data.shape[0],-1)
        # or data = data.reshape(data.shape[0], -1)
        #print(data.shape, targets.shape)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad() # VERY IMPORTANT
        loss.backward()
        
        # Gradient Descent/ Adam step
        optimizer.step()
        
        
        

In [63]:
# check accuracy on train and test
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval() # Very IMPORTANT
    
    with torch.no_grad(): # unnecessary to calculate gradients after training
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0],-1)
            scores = model(x) # [64x10]
            _, preds = scores.max(1) # along 2nd dimension i.e. 1
            num_correct+=(preds==y).sum()
            num_samples += preds.shape[0]
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)*100/float(num_samples):.2f}')

In [64]:
check_accuracy(train_loader,model=model)

Got 56790/ 60000 with accuracy 94.65


In [61]:
check_accuracy(test_loader,model=model)

Got 9425/ 10000 with accuracy 94.25


In [66]:
# check accuracy on train and test
def check_accuracy(loader,model):
    if loader.dataset.train:
        print('Checking on training data')
    else:
        print('Checking on test data')
    num_correct = 0
    num_samples = 0
    model.eval() # Very IMPORTANT
    
    with torch.no_grad(): # unnecessary to calculate gradients after training
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)
            x = x.reshape(x.shape[0],-1)
            scores = model(x) # [64x10]
            _, preds = scores.max(1) # along 2nd dimension i.e. 1
            num_correct+=(preds==y).sum()
            num_samples += preds.shape[0]
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)*100/float(num_samples):.2f}')
    model.train()

In [67]:
check_accuracy(train_loader,model=model)

Checking on training data
Got 56790/ 60000 with accuracy 94.65


In [62]:
# Troubleshooting (ME)
test_loader.dataset


Dataset MNIST
    Number of datapoints: 10000
    Root location: dataset/
    Split: Test
    StandardTransform
Transform: ToTensor()